In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import pickle
import random
random.seed(42)
np.random.seed(42)

In [3]:
with open("../ressources/automode_evaluated_concat_s14s15s16_n300_24-12-18.pickle", "rb") as file:
    original_df = pickle.load(file)

In [4]:
from pipeline import CustomSFTTrainer

sft_trainer = CustomSFTTrainer()
trained_model, hf_sft_trainer, dataset_train = sft_trainer.train()


/home/jj/miniforge3/envs/automode_llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.
`low_cpu_mem_usage` was None, now default to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 6/6 [00:58<00:00,  9.78s/it]
/home/jj/miniforge3/envs/automode_llm/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Map: 100%|██████████| 2/2 [00:00<00:00, 539.98 examples/s]


Epoch,Training Loss,Validation Loss
1,No log,2.460001
2,No log,2.388341
3,No log,2.317997
4,No log,2.258613
5,No log,2.207684
6,No log,2.164198
7,1.991100,2.128359
8,1.991100,2.101520
9,1.991100,2.083606
10,1.991100,2.073671


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

In [5]:
from pipeline import CustomInference

generated_text = CustomInference(model=trained_model,tokenizer=sft_trainer.tokenizer).inference()
generated_text

"<s>[INST] The environment features a rectangle with dimensions 5.57 x 2.19 x 1.75.There are the following lights in the arena: ((-0.25, 0.57)). 16 robots are evenly spaced around the central point, spanning a radius of 0.96 m. In the surroundings, there exists a circle at [-0.71, 0.76] with a radius of 0.25 meters, exhibiting a white color, and another at [0.78, -1.25] with a radius of 0.27 meters in black. The objective for the robots is to transfer items from the white initial location to the black circle. \nGenerate the behavior tree that achieves the objective of this mission.[/INST] To generate the behavior tree that achieves the objective of this mission, we need to define the tasks and sub-tasks that the robots need to perform. Here's a possible behavior tree:\n\n1. Root Node:\n\t* Initialize\n\t* Move to initial position\n\t* Pick up item\n\t* Move to target position\n\t* Drop item\n\t* Finish\n\n2. Initialize:\n\t* Set initial position\n\t* Set target position\n\t* Set item p

In [6]:
from pipeline.utils import DEFAULT_GENERATE_PROMPT

DEFAULT_GENERATE_PROMPT("test", "", sft_trainer)

'<s>[INST] test\nGenerate the behavior tree that achieves the objective of this mission.[/INST] </s>'

In [7]:
from pipeline import CustomDPOTrainer

dpo_trainer = CustomDPOTrainer()
dpo_trained_model, hf_dpo_trainer, dataset_train = dpo_trainer.train()

`low_cpu_mem_usage` was None, now default to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 6/6 [00:56<00:00,  9.46s/it]


TypeError: PeftModel.add_adapter() missing 1 required positional argument: 'peft_config'

In [ ]:
import random
import numpy as np
random.seed(42)
np.random.seed(42)

In [ ]:
generated_text = CustomInference(model=dpo_trained_model,tokenizer=dpo_trainer.tokenizer).inference()
generated_text

In [ ]:
dpo_trainer = CustomDPOTrainer()
random.seed(42)
np.random.seed(42)

generated_text = CustomInference(model=dpo_trainer.model,tokenizer=dpo_trainer.tokenizer).inference()
generated_text

In [41]:
model = sft_trainer.model

In [46]:
from pipeline.utils import load_default_model


model, tokenizer, lora, bnb = load_default_model()

`low_cpu_mem_usage` was None, now default to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 6/6 [00:58<00:00,  9.73s/it]


In [47]:
model.add_adapter("newadapter", lora)

In [ ]:
model.peft_config.keys()

dict_keys(['default', 'newadapter'])

In [ ]:
PeftModel(sft_trainer.model.base_model.config, sft_trainer.model.base_model)

AttributeError: 'MistralForCausalLM' object has no attribute 'peft_type'

In [12]:
from peft import PeftModel

PeftModel.from_pretrained(sft_trainer.model,"test")

ValueError: Can't find 'adapter_config.json' at 'test'